#Text Preprocessing

На цьому етапі виконується попередня обробка текстових даних для подальшого використання в моделях типу TF-IDF.

З огляду на те, що питання в датасеті є відносно короткими, а словниковий склад — досить різноманітним, було обрано саме методTF-IDF для векторизації тексту.

Даний підхід дозволяє зменшити вплив загальних слів та підкреслити інформативні терміни, зберігаючи при цьому простоту реалізації та ефективність обчислень.


In [27]:
import re
import pandas as pd
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_df = pd.read_csv("/content/drive/MyDrive/Final_Project/quora_question_pairs_train_.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Final_Project/quora_question_pairs_test.csv")
pd.set_option('display.max_colwidth', None)

In [4]:
train_df.shape, test_df.shape

((323432, 6), (80858, 6))

In [5]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,332278,459256,459257,The Iliad and the Odyssey in the Greek culture?,How do I prove that the pairs of three independent variables is also independent?,0
1,196656,297402,297403,What is practical management and what is strategic management?,What are the practical aspects of strategic management?,0
2,113125,184949,184950,How useful is MakeUseOf Answers?,"Is there any Q&A site that is not Yahoo answers, where hate speech is allowed?",0
3,266232,101283,163744,Which is the best place to reside in India and Why?,Which ia the best place to visit in India?,0
4,122738,17811,27517,Why do so many people ask questions on Quora that can be easily answered by any number of legitimate sources on the Web? Have they not heard of Google or Bing?,Why don't many people posting questions on Quora check Google first?,1


In [6]:
test_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,305985,429434,429435,Why is beef banned in India and not pork as well?,Is beef banned in india?,0
1,5193,10230,10231,At what valuation did Homejoy raise money in December of 2013?,"Should a wealthy founder self-fund his second startup then raise money at high valuation after getting traction, or raise money at low valuation before any traction?",0
2,123326,199422,199423,How do we judge?,How do I judge my love?,0
3,368557,327674,498931,Are Adderall and meth the same?,Are concerta and meth test the same?,0
4,369226,499645,499646,"If you had internet access to only one site for the rest of your life, which site would you pick?",Why is there .co.uk for British internet sites but only .fr for French ones?,0


In [7]:
test_df.isna().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [8]:
train_df.isna().sum()

,0
id,0
qid1,0
qid2,0
question1,1
question2,2
is_duplicate,0


In [9]:
train_df["question1"] = train_df["question1"].fillna("")
train_df["question2"] = train_df["question2"].fillna("")

In [10]:
train_df.isna().sum()

,0
id,0
qid1,0
qid2,0
question1,0
question2,0
is_duplicate,0


In [11]:
stemmer = SnowballStemmer("english")

In [12]:
def preprocess(text):
    text = text.lower()
    tokens = re.findall(r"[a-z]+", text)

    tokens_stemmed = []

    for t in tokens:
        stemmed_word = stemmer.stem(t)
        tokens_stemmed.append(stemmed_word)

    return tokens_stemmed


#Feature Extraction

Для перетворення текстових даних у числові ознаки використовуємо TF-IDF векторизацію з попередньою обробкою тексту.

Це дозволяє враховувати інформативність слів та ефективно працювати з текстовими даними великого обсягу

In [13]:
vectorizer_concat = TfidfVectorizer(
    tokenizer=preprocess,
    token_pattern=None,
    ngram_range=(1, 2),
    max_features=50_000
)

In [14]:
train_texts = train_df["question1"] + " " + train_df["question2"]
test_texts  = test_df["question1"] + " " + test_df["question2"]


In [15]:
X_train = vectorizer_concat.fit_transform(train_texts)
X_test  = vectorizer_concat.transform(test_texts)

print(X_train.shape)
print(X_test.shape)


(323432, 50000)
(80858, 50000)


#Text Similarity

Для оцінки схожості між парами питань було розглянуто підхід порівняння їх векторних представлень.

Основною метрикою обрано косинусну схожість.
Вона дозволяє оцінити ступінь семантичної близькості між двома питаннями незалежно від їх довжини.

In [16]:
vectorizer_pair = TfidfVectorizer(
    tokenizer=preprocess,
    token_pattern=None,
    ngram_range=(1, 2),
    max_features=50_000
)

In [17]:
X_q1_train = vectorizer_pair.fit_transform(train_df["question1"])
X_q2_train = vectorizer_pair.transform(train_df["question2"])

X_q1_test = vectorizer_pair.transform(test_df["question1"])
X_q2_test = vectorizer_pair.transform(test_df["question2"])


In [18]:
cos_train = np.array(X_q1_train.multiply(X_q2_train).sum(axis=1)).ravel()
cos_test  = np.array(X_q1_test.multiply(X_q2_test).sum(axis=1)).ravel()

In [19]:
cos_train.shape

(323432,)

In [20]:
cos_test.shape

(80858,)

In [21]:
X_train_cos = cos_train.reshape(-1, 1)
X_test_cos  = cos_test.reshape(-1, 1)

print("X_train_cos:", X_train_cos.shape)
print("X_test_cos:", X_test_cos.shape)

X_train_cos: (323432, 1)
X_test_cos: (80858, 1)


In [22]:
y_train = train_df["is_duplicate"]
y_test  = test_df["is_duplicate"]

#Base Model

У базовій моделі використовуємо лише одну фічу - значення косинусної схожості між TF-IDF представленнями пари питань, та цільову змінну is_duplicate

In [24]:
X_train = X_train_cos
X_test  = X_test_cos

y_train = train_df["is_duplicate"]
y_test  = test_df["is_duplicate"]


In [25]:
base_model = LogisticRegression(
    max_iter=1000,
    solver="liblinear"
)

base_model.fit(X_train, y_train)


LogisticRegression(max_iter=1000, solver='liblinear')

In [26]:
y_pred = base_model.predict(X_test)

In [28]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))

print("\nClassification report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.6240693561552351
F1-score: 0.32259933590355

Classification report:
              precision    recall  f1-score   support

           0       0.66      0.85      0.74     51005
           1       0.48      0.24      0.32     29853

    accuracy                           0.62     80858
   macro avg       0.57      0.54      0.53     80858
weighted avg       0.59      0.62      0.59     80858



In [29]:
print("Model coefficient:", base_model.coef_)
print("Model intercept:", base_model.intercept_)


Model coefficient: [[2.12467161]]
Model intercept: [-1.46976351]


In [30]:
y_probabilities = base_model.predict_proba(X_test)[:, 1]


In [31]:
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [32]:
for threshold in thresholds:

    predictions = []
    for p in y_probabilities:
        if p >= threshold:
            predictions.append(1)
        else:
            predictions.append(0)

    f1 = f1_score(y_test, predictions)

    print("Threshold:", threshold, "→ F1-score:", f1)


Threshold: 0.1 → F1-score: 0.5392960049136942
Threshold: 0.2 → F1-score: 0.5743838664008553
Threshold: 0.3 → F1-score: 0.5970910255304999
Threshold: 0.4 → F1-score: 0.4991181361597769
Threshold: 0.5 → F1-score: 0.32259933590355
Threshold: 0.6 → F1-score: 0.11723920579892846
Threshold: 0.7 → F1-score: 0.0
Threshold: 0.8 → F1-score: 0.0
Threshold: 0.9 → F1-score: 0.0


##Висновок

Базова модель, побудована з використанням логістичної регресії та єдиної ознаки — косинусної схожості між TF-IDF представленнями пари запитань, продемонструвала обмежену якість при стандартному порозі прийняття рішення 0.5 (F1-score ≈ 0.32).

Це свідчить про те, що модель є занадто обережною у передбаченні класу дублікатів.

Підбір порогу прийняття рішення показав значний вплив на якість моделі: при зменшенні порогу до 0.3 F1-score зріс до ≈ 0.60.